In [11]:
import torch
import torchvision
import cv2
import pycocotools
from pycocotools.coco import COCO

print(f"PyTorch: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

PyTorch: 2.6.0+cu124
GPU Available: True
GPU Name: Tesla P100-PCIE-16GB


In [12]:
# Clone the official Mask Scoring R-CNN repo
!git clone https://github.com/zjhuang22/maskscoring_rcnn.git

# Navigate into it
import os
os.chdir('/kaggle/working/maskscoring_rcnn')

# List contents to see what's available
print("\nRepository contents:")
!ls -la

print("\nLooking for key directories:")
!ls -d */ 2>/dev/null || echo "No subdirectories found"

Cloning into 'maskscoring_rcnn'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 401 (delta 51), reused 48 (delta 48), pack-reused 324 (from 1)
Receiving objects: 100% (401/401), 5.21 MiB | 22.70 MiB/s, done.
Resolving deltas: 100% (129/129), done.

Repository contents:
total 60
drwxr-xr-x 11 root root 4096 Nov  9 04:18 .
drwxr-xr-x  4 root root 4096 Nov  9 02:54 ..
drwxr-xr-x  4 root root 4096 Nov  9 02:54 configs
drwxr-xr-x  2 root root 4096 Nov  9 02:54 demo
drwxr-xr-x  2 root root 4096 Nov  9 02:54 docker
drwxr-xr-x  8 root root 4096 Nov  9 02:54 .git
-rw-r--r--  1 root root 1602 Nov  9 02:54 INSTALL.md
-rw-r--r--  1 root root 1070 Nov  9 02:54 LICENSE
drwxr-xr-x 11 root root 4096 Nov  9 02:54 maskrcnn_benchmark
drwxr-xr-x  9 root root 4096 Nov  9 04:18 maskscoring_rcnn
-rw-r--r--  1 root root 3950 Nov  9 02:54 README.md
drwxr-xr-x  2 root root 4096 Nov  9 03:39 results
-rw-r--r--

In [13]:
import os
import sys

os.chdir('/kaggle/working/maskscoring_rcnn')

# Add the repo to Python path instead of installing
sys.path.insert(0, '/kaggle/working/maskscoring_rcnn')

print("✓ Added MS R-CNN to Python path")
print(f"Python path: {sys.path[0]}")

✓ Added MS R-CNN to Python path
Python path: /kaggle/working/maskscoring_rcnn


In [14]:
import os
import sys

os.chdir('/kaggle/working/maskscoring_rcnn')

# Add repo to Python path
sys.path.insert(0, '/kaggle/working/maskscoring_rcnn')

# Set environment variable
os.environ['PYTHONPATH'] = '/kaggle/working/maskscoring_rcnn'

print("Testing direct imports without installation...")
print("=" * 60)

# Try importing from the source directly
try:
    import maskrcnn_benchmark
    print(f"✓ maskrcnn_benchmark found at: {maskrcnn_benchmark.__file__}")
except ImportError:
    print("✗ Direct import failed, trying alternatives...")

# Check if files exist
maskrcnn_path = '/kaggle/working/maskscoring_rcnn/maskrcnn_benchmark'
if os.path.exists(maskrcnn_path):
    print(f"✓ maskrcnn_benchmark directory exists")
    init_file = os.path.join(maskrcnn_path, '__init__.py')
    if os.path.exists(init_file):
        print(f"✓ __init__.py found")
    else:
        print("✗ __init__.py missing - creating it...")
        # Create missing __init__.py if needed
        with open(init_file, 'w') as f:
            f.write("# Mask R-CNN Benchmark\n")
else:
    print("✗ maskrcnn_benchmark directory not found!")

print("\n✓ Setup complete")


Testing direct imports without installation...
✓ maskrcnn_benchmark found at: /kaggle/working/maskscoring_rcnn/maskrcnn_benchmark/__init__.py
✓ maskrcnn_benchmark directory exists
✓ __init__.py found

✓ Setup complete


In [15]:
# Just install the dependencies, not the package itself
!pip install -q yacs
!pip install -q tensorboard

print("✓ Dependencies installed")

✓ Dependencies installed


In [16]:
import os
os.chdir('/kaggle/working/maskscoring_rcnn')

# Check if test script exists
test_script = 'tools/test_net.py'
if os.path.exists(test_script):
    print(f"✓ Found: {test_script}")
    
    # Read first few lines to understand structure
    with open(test_script, 'r') as f:
        lines = f.readlines()[:20]
        print("\nFirst 20 lines of test_net.py:")
        print("=" * 60)
        for i, line in enumerate(lines, 1):
            print(f"{i:2d}: {line.rstrip()}")
else:
    print(f"✗ {test_script} not found")
    
# List all tools
print("\n\nAvailable scripts in tools/:")
!ls -lh tools/

✓ Found: tools/test_net.py

First 20 lines of test_net.py:
 1: # Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
 2: # Set up custom environment before nearly anything else is imported
 3: # NOTE: this should be the first import (no not reorder)
 4: from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip
 5: 
 6: import argparse
 7: import os
 8: 
 9: import torch
10: from maskrcnn_benchmark.config import cfg
11: from maskrcnn_benchmark.data import make_data_loader
12: from maskrcnn_benchmark.engine.inference import inference
13: from maskrcnn_benchmark.modeling.detector import build_detection_model
14: from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
15: from maskrcnn_benchmark.utils.collect_env import collect_env_info
16: from maskrcnn_benchmark.utils.comm import synchronize, get_rank
17: from maskrcnn_benchmark.utils.logger import setup_logger
18: from maskrcnn_benchmark.utils.miscellaneous import mkdir
19: 
20: 


In [17]:
import os
import sys

os.chdir('/kaggle/working/maskscoring_rcnn')

# Create a compatibility module for torch._six
compat_code = '''
"""PyTorch 2.x compatibility shim"""
import sys
import torch

# PyTorch 2.x removed torch._six, so we create a replacement
if not hasattr(torch, '_six'):
    class _six:
        string_classes = (str,)
        
        @staticmethod
        def string_types(*args):
            return str
    
    torch._six = _six

print("✓ PyTorch compatibility shim loaded")
'''

# Create compat module
compat_path = 'maskrcnn_benchmark/torch_compat.py'
with open(compat_path, 'w') as f:
    f.write(compat_code)

print(f"✓ Created compatibility module at {compat_path}")

# Update __init__.py to import compat first
init_path = 'maskrcnn_benchmark/__init__.py'
with open(init_path, 'r') as f:
    init_content = f.read()

# Add compat import at the very beginning if not already there
if 'torch_compat' not in init_content:
    init_content = 'from . import torch_compat\n' + init_content
    with open(init_path, 'w') as f:
        f.write(init_content)
    print(f"✓ Updated {init_path} with compat import")
else:
    print(f"✓ torch_compat already imported in {init_path}")


✓ Created compatibility module at maskrcnn_benchmark/torch_compat.py
✓ Updated maskrcnn_benchmark/__init__.py with compat import


In [18]:
import os

print("Checking available COCO dataset paths on Kaggle...")
print("=" * 60)

# Check what's available in /kaggle/input
kaggle_input = '/kaggle/input'
print(f"\nContents of {kaggle_input}:")
for item in os.listdir(kaggle_input):
    path = os.path.join(kaggle_input, item)
    if os.path.isdir(path):
        print(f"  📁 {item}")

# Look for COCO specifically
print("\nSearching for COCO dataset...")
for item in os.listdir(kaggle_input):
    if 'coco' in item.lower():
        coco_path = os.path.join(kaggle_input, item)
        print(f"\n✓ Found: {coco_path}")
        
        # Check contents
        try:
            contents = os.listdir(coco_path)
            print(f"  Contents: {contents[:5]}...")  # First 5 items
            
            # Check for annotations
            if 'annotations' in contents:
                ann_path = os.path.join(coco_path, 'annotations')
                anns = os.listdir(ann_path)
                print(f"  Annotations: {anns[:3]}")
        except:
            pass


Checking available COCO dataset paths on Kaggle...

Contents of /kaggle/input:
  📁 coco-2017-dataset

Searching for COCO dataset...

✓ Found: /kaggle/input/coco-2017-dataset
  Contents: ['coco2017']...


In [19]:
import os

coco_path = '/kaggle/input/coco-2017-dataset/coco2017'

print("Checking COCO2017 structure...")
print("=" * 60)

if os.path.exists(coco_path):
    contents = os.listdir(coco_path)
    print(f"\nContents of {coco_path}:")
    for item in contents:
        item_path = os.path.join(coco_path, item)
        if os.path.isdir(item_path):
            count = len(os.listdir(item_path))
            print(f"  📁 {item}: {count} items")
        else:
            size = os.path.getsize(item_path) / (1024**2)
            print(f"  📄 {item}: {size:.1f} MB")
else:
    print(f"✗ Path not found: {coco_path}")
    
    # Check parent
    parent = '/kaggle/input/coco-2017-dataset'
    print(f"\nTrying parent: {parent}")
    if os.path.exists(parent):
        print(f"Contents:")
        for item in os.listdir(parent):
            print(f"  - {item}")


Checking COCO2017 structure...

Contents of /kaggle/input/coco-2017-dataset/coco2017:
  📁 val2017: 5000 items
  📁 annotations: 6 items
  📁 test2017: 40670 items
  📁 train2017: 118287 items


In [20]:
import os
import sys
import torch
import cv2
import numpy as np
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection import maskrcnn_resnet50_fpn
import json
import time

os.chdir('/kaggle/working/maskscoring_rcnn')
sys.path.insert(0, '/kaggle/working/maskscoring_rcnn')

print("=" * 60)
print("MS R-CNN BASELINE - TORCHVISION")
print("=" * 60)

# Correct paths
coco_ann_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
coco_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/val2017'

print(f"\n✓ COCO annotations: {coco_ann_file}")
print(f"✓ COCO images: {coco_images_dir}")

# Load COCO
print("\nLoading COCO validation set...")
coco_gt = COCO(coco_ann_file)
img_ids = sorted(coco_gt.getImgIds())
print(f"✓ Loaded {len(img_ids)} images")

# Load model
print("\nLoading Mask R-CNN (ResNet-50-FPN)...")
model = maskrcnn_resnet50_fpn(pretrained=True)
model.eval()
model.cuda()
print("✓ Model loaded and moved to GPU")

# Inference
print(f"\nRunning inference on validation set...")
print("=" * 60)

predictions = []
times = []
processed = 0

for idx, img_id in enumerate(img_ids):
    img_info = coco_gt.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_dir, img_info['file_name'])
    
    if not os.path.exists(img_path):
        continue
    
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        continue
        
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = torch.from_numpy(img_rgb).float().cuda() / 255.0
    img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0)
    
    # Inference
    start = time.time()
    with torch.no_grad():
        outputs = model([img_tensor.squeeze(0)])
    times.append(time.time() - start)
    
    # Extract predictions
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()
    
    # Convert to COCO format
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        
        predictions.append({
            'image_id': int(img_id),
            'category_id': int(labels[i]),
            'bbox': [float(x1), float(y1), float(x2-x1), float(y2-y1)],
            'score': float(scores[i])
        })
    
    processed += 1
    
    if (processed) % 500 == 0:
        avg_time = np.mean(times[-500:])
        remaining = len(img_ids) - processed
        eta_secs = remaining * avg_time
        print(f"  {processed}/{len(img_ids)} - {avg_time:.3f}s/img - ETA: {eta_secs/60:.1f} mins")

print(f"\n" + "=" * 60)
print(f"✓ Inference complete!")
print(f"✓ Processed: {processed} images")
print(f"✓ Total predictions: {len(predictions)}")
print(f"✓ Average time: {np.mean(times):.3f}s per image")
print(f"✓ Throughput: {1/np.mean(times):.2f} FPS")

# Save predictions
os.makedirs('results', exist_ok=True)
pred_file = 'results/predictions.json'
with open(pred_file, 'w') as f:
    json.dump(predictions, f)
print(f"\n✓ Predictions saved to: {pred_file}")


MS R-CNN BASELINE - TORCHVISION

✓ COCO annotations: /kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json
✓ COCO images: /kaggle/input/coco-2017-dataset/coco2017/val2017

Loading COCO validation set...
loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
✓ Loaded 5000 images

Loading Mask R-CNN (ResNet-50-FPN)...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 215MB/s] 


✓ Model loaded and moved to GPU

Running inference on validation set...
  500/5000 - 0.083s/img - ETA: 6.2 mins
  1000/5000 - 0.085s/img - ETA: 5.7 mins
  1500/5000 - 0.084s/img - ETA: 4.9 mins
  2000/5000 - 0.083s/img - ETA: 4.1 mins
  2500/5000 - 0.083s/img - ETA: 3.5 mins
  3000/5000 - 0.084s/img - ETA: 2.8 mins
  3500/5000 - 0.084s/img - ETA: 2.1 mins
  4000/5000 - 0.083s/img - ETA: 1.4 mins
  4500/5000 - 0.084s/img - ETA: 0.7 mins
  5000/5000 - 0.083s/img - ETA: 0.0 mins

✓ Inference complete!
✓ Processed: 5000 images
✓ Total predictions: 170603
✓ Average time: 0.083s per image
✓ Throughput: 11.98 FPS

✓ Predictions saved to: results/predictions.json


In [21]:
import os
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

os.chdir('/kaggle/working/maskscoring_rcnn')

print("=" * 60)
print("EVALUATING BASELINE RESULTS")
print("=" * 60)

# Load ground truth
coco_ann_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
coco_gt = COCO(coco_ann_file)

# Load predictions
print("\nLoading predictions...")
pred_file = 'results/predictions.json'
coco_dt = coco_gt.loadRes(pred_file)

# Evaluate
print("Computing COCO metrics (this may take a few minutes)...")
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Extract results
ap_50_95 = coco_eval.stats[0]
ap_50 = coco_eval.stats[1]
ap_75 = coco_eval.stats[2]
ap_small = coco_eval.stats[3]
ap_medium = coco_eval.stats[4]
ap_large = coco_eval.stats[5]

print("\n" + "=" * 60)
print("BASELINE RESULTS SUMMARY")
print("=" * 60)
print(f"\nModel: Mask R-CNN (ResNet-50-FPN)")
print(f"Framework: Torchvision Pretrained")
print(f"Dataset: COCO 2017 Validation ({len(coco_gt.getImgIds())} images)")
print(f"\nBounding Box Metrics:")
print(f"  AP@IoU=0.50:0.95: {ap_50_95:.4f}")
print(f"  AP@IoU=0.50:     {ap_50:.4f}")
print(f"  AP@IoU=0.75:     {ap_75:.4f}")
print(f"  AP_small:        {ap_small:.4f}")
print(f"  AP_medium:       {ap_medium:.4f}")
print(f"  AP_large:        {ap_large:.4f}")

# Save results
baseline_results = {
    'model': 'Mask R-CNN (ResNet-50-FPN)',
    'framework': 'Torchvision',
    'dataset': 'COCO 2017 Validation',
    'num_images': len(coco_gt.getImgIds()),
    'num_predictions': len(json.load(open(pred_file))),
    'metrics': {
        'AP@0.50:0.95': float(ap_50_95),
        'AP@0.50': float(ap_50),
        'AP@0.75': float(ap_75),
        'AP_small': float(ap_small),
        'AP_medium': float(ap_medium),
        'AP_large': float(ap_large),
    }
}

results_file = 'results/baseline_results.json'
with open(results_file, 'w') as f:
    json.dump(baseline_results, f, indent=2)

print(f"\n✓ Results saved to: {results_file}")
print("\n" + "=" * 60)
print("✓ BASELINE ESTABLISHED!")
print("=" * 60)


EVALUATING BASELINE RESULTS
loading annotations into memory...
Done (t=0.44s)
creating index...
index created!

Loading predictions...
Loading and preparing results...
DONE (t=1.61s)
creating index...
index created!
Computing COCO metrics (this may take a few minutes)...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=23.51s).
Accumulating evaluation results...
DONE (t=3.87s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.379
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.411
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.414
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.493
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.313
 Average Recall     (AR) @[ IoU=0.50

In [22]:
from torchvision.models.detection import *
import inspect

print("Available Mask R-CNN models in this session:")
print("=" * 60)

# Get all available detection models
available = []
for name in dir():
    if 'maskrcnn' in name.lower():
        available.append(name)
        print(f"  ✓ {name}")

if available:
    print(f"\n✓ Total available: {len(available)} models")
else:
    print("\n No Mask R-CNN models found")



Available Mask R-CNN models in this session:
  ✓ MaskRCNN
  ✓ MaskRCNN_ResNet50_FPN_V2_Weights
  ✓ MaskRCNN_ResNet50_FPN_Weights
  ✓ maskrcnn_benchmark
  ✓ maskrcnn_path
  ✓ maskrcnn_resnet50_fpn
  ✓ maskrcnn_resnet50_fpn_v2

✓ Total available: 7 models


In [23]:
import os
import sys
import torch
import cv2
import numpy as np
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2
import json
import time

os.chdir('/kaggle/working/maskscoring_rcnn')
sys.path.insert(0, '/kaggle/working/maskscoring_rcnn')

print("=" * 60)
print("IMPROVEMENT C: ResNet-50 V2 (Improved Model)")
print("=" * 60)

# COCO paths
coco_ann_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
coco_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/val2017'

print(f"\n✓ COCO annotations: {coco_ann_file}")
print(f"✓ COCO images: {coco_images_dir}")

# Load COCO
print("\nLoading COCO validation set...")
coco_gt = COCO(coco_ann_file)
img_ids = sorted(coco_gt.getImgIds())
print(f"✓ Loaded {len(img_ids)} images")

# Load ResNet-50 V2 model (IMPROVED VERSION!)
print("\nLoading Mask R-CNN with ResNet-50-FPN V2 (improved backbone)...")
model = maskrcnn_resnet50_fpn_v2(pretrained=True)
model.eval()
model.cuda()
print("✓ Model loaded and moved to GPU")

# Inference
print(f"\nRunning inference on validation set...")
print("=" * 60)

predictions = []
times = []
processed = 0

for idx, img_id in enumerate(img_ids):
    img_info = coco_gt.loadImgs(img_id)[0]
    img_path = os.path.join(coco_images_dir, img_info['file_name'])
    
    if not os.path.exists(img_path):
        continue
    
    # Load image
    img = cv2.imread(img_path)
    if img is None:
        continue
        
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = torch.from_numpy(img_rgb).float().cuda() / 255.0
    img_tensor = img_tensor.permute(2, 0, 1).unsqueeze(0)
    
    # Inference
    start = time.time()
    with torch.no_grad():
        outputs = model([img_tensor.squeeze(0)])
    times.append(time.time() - start)
    
    # Extract predictions
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()
    
    # Convert to COCO format
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        
        predictions.append({
            'image_id': int(img_id),
            'category_id': int(labels[i]),
            'bbox': [float(x1), float(y1), float(x2-x1), float(y2-y1)],
            'score': float(scores[i])
        })
    
    processed += 1
    
    if (processed) % 500 == 0:
        avg_time = np.mean(times[-500:])
        remaining = len(img_ids) - processed
        eta_secs = remaining * avg_time
        print(f"  {processed}/{len(img_ids)} - {avg_time:.3f}s/img - ETA: {eta_secs/60:.1f} mins")

print(f"\n" + "=" * 60)
print(f"✓ Inference complete!")
print(f"✓ Processed: {processed} images")
print(f"✓ Total predictions: {len(predictions)}")
print(f"✓ Average time: {np.mean(times):.3f}s per image")
print(f"✓ Throughput: {1/np.mean(times):.2f} FPS")

# Save predictions
os.makedirs('results', exist_ok=True)
pred_file = 'results/resnet50v2_predictions.json'
with open(pred_file, 'w') as f:
    json.dump(predictions, f)
print(f"\n✓ Predictions saved to: {pred_file}")

IMPROVEMENT C: ResNet-50 V2 (Improved Model)

✓ COCO annotations: /kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json
✓ COCO images: /kaggle/input/coco-2017-dataset/coco2017/val2017

Loading COCO validation set...
loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
✓ Loaded 5000 images

Loading Mask R-CNN with ResNet-50-FPN V2 (improved backbone)...
✓ Model loaded and moved to GPU

Running inference on validation set...
  500/5000 - 0.123s/img - ETA: 9.2 mins
  1000/5000 - 0.124s/img - ETA: 8.2 mins
  1500/5000 - 0.124s/img - ETA: 7.2 mins
  2000/5000 - 0.123s/img - ETA: 6.2 mins
  2500/5000 - 0.123s/img - ETA: 5.1 mins
  3000/5000 - 0.124s/img - ETA: 4.1 mins
  3500/5000 - 0.124s/img - ETA: 3.1 mins
  4000/5000 - 0.124s/img - ETA: 2.1 mins
  4500/5000 - 0.126s/img - ETA: 1.0 mins
  5000/5000 - 0.124s/img - ETA: 0.0 mins

✓ Inference complete!
✓ Processed: 5000 images
✓ Total predictions: 160837
✓ Average time: 0.124s per image
✓ T

In [24]:
import os
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

os.chdir('/kaggle/working/maskscoring_rcnn')

print("=" * 60)
print("EVALUATING RESNET-50 V2 RESULTS")
print("=" * 60)

# Load baseline results from file
baseline_file = 'results/baseline_results.json'
if os.path.exists(baseline_file):
    with open(baseline_file, 'r') as f:
        baseline_data = json.load(f)
    
    baseline_ap = baseline_data['metrics']['AP@0.50:0.95']
    baseline_ap_50 = baseline_data['metrics']['AP@0.50']
    baseline_ap_75 = baseline_data['metrics']['AP@0.75']
    baseline_ap_small = baseline_data['metrics']['AP_small']
    baseline_ap_medium = baseline_data['metrics']['AP_medium']
    baseline_ap_large = baseline_data['metrics']['AP_large']
    
    print(f"✓ Loaded baseline from: {baseline_file}")
    print(f"  Baseline AP: {baseline_ap*100:.2f}%")
else:
    print(f"✗ Baseline file not found: {baseline_file}")
    print("Cannot proceed without baseline")
    exit()

# Load ground truth
coco_ann_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_val2017.json'
coco_gt = COCO(coco_ann_file)

# Load predictions
print("\nLoading predictions...")
pred_file = 'results/resnet50v2_predictions.json'
coco_dt = coco_gt.loadRes(pred_file)

# Evaluate
print("Computing COCO metrics (this may take a few minutes)...")
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Extract results
ap_50_95 = coco_eval.stats[0]
ap_50 = coco_eval.stats[1]
ap_75 = coco_eval.stats[2]
ap_small = coco_eval.stats[3]
ap_medium = coco_eval.stats[4]
ap_large = coco_eval.stats[5]

print("\n" + "=" * 60)
print("RESNET-50 V2 vs BASELINE COMPARISON")
print("=" * 60)

print(f"\n{'Metric':<20} {'ResNet-50 V2':<15} {'Baseline':<15} {'Change':<10}")
print("-" * 70)
print(f"{'AP@0.50:0.95':<20} {ap_50_95*100:>6.2f}% {baseline_ap*100:>14.2f}% {(ap_50_95*100-baseline_ap*100):>9.2f}%")
print(f"{'AP@0.50':<20} {ap_50*100:>6.2f}% {baseline_ap_50*100:>14.2f}% {(ap_50*100-baseline_ap_50*100):>9.2f}%")
print(f"{'AP@0.75':<20} {ap_75*100:>6.2f}% {baseline_ap_75*100:>14.2f}% {(ap_75*100-baseline_ap_75*100):>9.2f}%")
print(f"{'AP_small':<20} {ap_small*100:>6.2f}% {baseline_ap_small*100:>14.2f}% {(ap_small*100-baseline_ap_small*100):>9.2f}%")
print(f"{'AP_medium':<20} {ap_medium*100:>6.2f}% {baseline_ap_medium*100:>14.2f}% {(ap_medium*100-baseline_ap_medium*100):>9.2f}%")
print(f"{'AP_large':<20} {ap_large*100:>6.2f}% {baseline_ap_large*100:>14.2f}% {(ap_large*100-baseline_ap_large*100):>9.2f}%")

# Save results
resnet50v2_results = {
    'improvement': 'ResNet-50 V2 (Improved Backbone)',
    'model': 'Mask R-CNN (ResNet-50-FPN V2)',
    'framework': 'Torchvision',
    'dataset': 'COCO 2017 Validation',
    'num_images': len(coco_gt.getImgIds()),
    'num_predictions': len(json.load(open(pred_file))),
    'inference_time': {
        'avg_per_image': 0.124,
        'throughput_fps': 8.06
    },
    'metrics': {
        'AP@0.50:0.95': float(ap_50_95),
        'AP@0.50': float(ap_50),
        'AP@0.75': float(ap_75),
        'AP_small': float(ap_small),
        'AP_medium': float(ap_medium),
        'AP_large': float(ap_large),
    },
    'comparison_with_baseline': {
        'baseline_model': baseline_data.get('model', 'Mask R-CNN (ResNet-50-FPN)'),
        'baseline_ap': float(baseline_ap),
        'new_ap': float(ap_50_95),
        'improvement_absolute': float(ap_50_95 - baseline_ap),
        'improvement_relative': float((ap_50_95 - baseline_ap) / baseline_ap * 100)
    }
}

results_file = 'results/resnet50v2_results.json'
with open(results_file, 'w') as f:
    json.dump(resnet50v2_results, f, indent=2)

print(f"\n✓ Results saved to: {results_file}")

print("\n" + "=" * 60)
print("IMPROVEMENT C - SUMMARY")
print("=" * 60)
print(f"Baseline (ResNet-50 V1):  {baseline_ap*100:.2f}% AP")
print(f"ResNet-50 V2:             {ap_50_95*100:.2f}% AP")
print(f"Absolute improvement:     {(ap_50_95-baseline_ap)*100:+.2f}%")
print(f"Relative improvement:     {((ap_50_95-baseline_ap)/baseline_ap)*100:+.2f}%")

if ap_50_95 > baseline_ap:
    print("\n IMPROVEMENT SUCCESSFUL!")
else:
    print("\n No improvement observed")

print("=" * 60)


EVALUATING RESNET-50 V2 RESULTS
✓ Loaded baseline from: results/baseline_results.json
  Baseline AP: 37.89%
loading annotations into memory...
Done (t=0.44s)
creating index...
index created!

Loading predictions...
Loading and preparing results...
DONE (t=0.66s)
creating index...
index created!
Computing COCO metrics (this may take a few minutes)...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=23.99s).
Accumulating evaluation results...
DONE (t=3.93s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.474
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.679
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.309
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.611
 Average Recall     (AR) @[ IoU=0.50